# BTC Stochastic + BTC/ETH Ratio Linear Combo

This notebook builds:
1. A **BTC stochastic long/flat model**.
2. A **BTC-vs-ETH ratio long/flat model** (based on spread z-score).
3. An optimized **linear combo** of both models.

Then it compares all strategies against **BTC** and **ETH** buy-and-hold, including Sharpe.


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path


In [ ]:
def load_crypto_data() -> pd.DataFrame:
    candidates = [
        Path("../Market Data/Crypto Data/cleaned_crypto_data.csv"),
        Path("Market Data/Crypto Data/cleaned_crypto_data.csv"),
    ]
    for p in candidates:
        if p.exists():
            df = pd.read_csv(p)
            break
    else:
        raise FileNotFoundError("cleaned_crypto_data.csv not found")

    df["Date"] = pd.to_datetime(df["Date"])
    df = df.sort_values("Date").set_index("Date")

    keep = ["BTC-USD_close", "ETH-USD_close", "BTC_REWARD", "COST_TO_MINE"]
    for c in keep:
        if c not in df.columns:
            raise KeyError(f"Missing required column: {c}")

    out = df[keep].rename(columns={
        "BTC-USD_close": "btc",
        "ETH-USD_close": "eth",
        "BTC_REWARD": "reward",
        "COST_TO_MINE": "cost_to_mine",
    }).dropna()

    return out


df = load_crypto_data()
df.head()


In [ ]:
TRADING_DAYS = 252


def sharpe_ratio(daily_returns: pd.Series, periods: int = TRADING_DAYS) -> float:
    r = daily_returns.dropna()
    if len(r) < 2:
        return np.nan
    vol = r.std(ddof=1)
    if vol == 0 or not np.isfinite(vol):
        return np.nan
    return float((r.mean() / vol) * np.sqrt(periods))


def max_drawdown(daily_returns: pd.Series) -> float:
    r = daily_returns.fillna(0.0)
    eq = (1 + r).cumprod()
    dd = eq / eq.cummax() - 1.0
    return float(dd.min())


def summarize(daily_returns: pd.Series, name: str) -> dict:
    r = daily_returns.dropna()
    eq = (1 + r).cumprod()
    total = float(eq.iloc[-1] - 1.0) if len(eq) else np.nan
    ann_ret = float((1 + total) ** (TRADING_DAYS / len(r)) - 1.0) if len(r) else np.nan
    ann_vol = float(r.std(ddof=1) * np.sqrt(TRADING_DAYS)) if len(r) > 1 else np.nan
    sh = sharpe_ratio(r)
    mdd = max_drawdown(r)
    calmar = float(ann_ret / abs(mdd)) if np.isfinite(ann_ret) and np.isfinite(mdd) and mdd < 0 else np.nan
    return {
        "strategy": name,
        "total_return": total,
        "annualized_return": ann_ret,
        "annualized_vol": ann_vol,
        "sharpe": sh,
        "max_drawdown": mdd,
        "calmar": calmar,
    }


In [ ]:
def build_stochastic_signal(price: pd.Series, k_window: int = 14, d_window: int = 3, oversold: float = 20.0, overbought: float = 80.0) -> pd.Series:
    low_k = price.rolling(k_window).min()
    high_k = price.rolling(k_window).max()
    pct_k = 100 * (price - low_k) / (high_k - low_k)
    pct_d = pct_k.rolling(d_window).mean()

    pos = pd.Series(index=price.index, data=0.0)
    in_pos = 0.0
    for i in range(len(price)):
        k = pct_k.iat[i]
        d = pct_d.iat[i]

        if not np.isfinite(k) or not np.isfinite(d):
            pos.iat[i] = in_pos
            continue

        if in_pos == 0.0:
            if (k > d) and (k < oversold):
                in_pos = 1.0
        else:
            if (k < d) and (k > overbought):
                in_pos = 0.0

        pos.iat[i] = in_pos

    return pos


def estimate_beta(log_btc: pd.Series, log_eth: pd.Series) -> float:
    x = log_eth.values
    y = log_btc.values
    x_mean = x.mean()
    y_mean = y.mean()
    denom = np.sum((x - x_mean) ** 2)
    if denom == 0:
        return 1.0
    return float(np.sum((x - x_mean) * (y - y_mean)) / denom)


def build_ratio_signal(btc: pd.Series, eth: pd.Series, lookback: int = 90, entry_z: float = 1.7, exit_z: float = 0.4) -> tuple[pd.Series, pd.Series]:
    log_btc = np.log(btc)
    log_eth = np.log(eth)
    beta = estimate_beta(log_btc.dropna(), log_eth.dropna())
    spread = log_btc - beta * log_eth

    m = spread.rolling(lookback).mean()
    s = spread.rolling(lookback).std(ddof=0).replace(0.0, np.nan)
    z = (spread - m) / s

    pos = pd.Series(index=spread.index, data=0.0)
    in_pos = 0.0
    for i in range(len(spread)):
        zi = z.iat[i]
        if not np.isfinite(zi):
            pos.iat[i] = in_pos
            continue

        if in_pos == 0.0 and zi < -entry_z:
            in_pos = 1.0
        elif in_pos == 1.0 and zi > -exit_z:
            in_pos = 0.0

        pos.iat[i] = in_pos

    return pos, spread


In [ ]:
def backtest_weighted_long(price: pd.Series, weight: pd.Series, vol_target: float = 0.20, vol_lb: int = 20, max_leverage: float = 1.5) -> pd.Series:
    ret = price.pct_change().fillna(0.0)
    w = weight.reindex(price.index).fillna(0.0).clip(lower=0.0)

    realized = ret.rolling(vol_lb, min_periods=max(5, vol_lb // 3)).std(ddof=0) * np.sqrt(TRADING_DAYS)
    scale = (vol_target / realized).replace([np.inf, -np.inf], np.nan).fillna(0.0)
    w = (w * scale).clip(upper=max_leverage)

    strat = w.shift(1).fillna(0.0) * ret
    return strat


def optimize_linear_combo(df: pd.DataFrame):
    """
    Two-stage search designed to improve on BTC stochastic Sharpe:
    1) Find best stochastic baseline Sharpe.
    2) Search ratio + linear blend and rank primarily by Sharpe uplift over baseline.
    """
    btc = df["btc"]
    eth = df["eth"]

    # Stage 1: best stochastic baseline
    stoch_rows = []
    best_stoch = {"sharpe": -np.inf, "params": None, "returns": None, "pos": None}
    for k_window in [8, 10, 14, 20, 28]:
        for d_window in [2, 3, 5, 8]:
            stoch_pos = build_stochastic_signal(btc, k_window=k_window, d_window=d_window)
            stoch_ret = backtest_weighted_long(btc, stoch_pos)
            sh = sharpe_ratio(stoch_ret)
            if not np.isfinite(sh):
                continue
            stoch_rows.append({"k_window": k_window, "d_window": d_window, "sharpe": float(sh)})
            if sh > best_stoch["sharpe"]:
                best_stoch = {
                    "sharpe": float(sh),
                    "params": {"k_window": k_window, "d_window": d_window},
                    "returns": stoch_ret,
                    "pos": stoch_pos,
                }

    if best_stoch["params"] is None:
        raise RuntimeError("No valid stochastic baseline found.")

    baseline_sharpe = best_stoch["sharpe"]

    # Stage 2: ratio model + combo optimization
    combo_rows = []
    best_combo = {"objective": -np.inf, "sharpe": -np.inf, "params": None, "returns": None, "weights": None, "uplift": np.nan}

    # keep stochastic fixed at best baseline; search whether ratio adds value
    stoch_pos = best_stoch["pos"]

    for lookback in [45, 60, 90, 120, 180, 240]:
        for entry_z in [1.0, 1.2, 1.5, 1.8, 2.1, 2.4]:
            for exit_z in [0.1, 0.2, 0.4, 0.6, 0.8]:
                if exit_z >= entry_z:
                    continue

                ratio_pos, _ = build_ratio_signal(btc, eth, lookback=lookback, entry_z=entry_z, exit_z=exit_z)
                ratio_ret = backtest_weighted_long(btc, ratio_pos)
                ratio_sh = sharpe_ratio(ratio_ret)

                # include 0 to allow fallback to pure stochastic; extend range so ratio can dominate if useful
                for w_stoch in np.linspace(0.5, 2.0, 16):
                    for w_ratio in np.linspace(0.0, 2.0, 21):
                        combo_weight = (w_stoch * stoch_pos + w_ratio * ratio_pos).clip(upper=2.5)
                        combo_ret = backtest_weighted_long(btc, combo_weight)
                        combo_sh = sharpe_ratio(combo_ret)
                        if not np.isfinite(combo_sh):
                            continue

                        mdd = max_drawdown(combo_ret)
                        uplift = combo_sh - baseline_sharpe

                        # Objective: prioritize sharpe uplift, then absolute sharpe, then drawdown control
                        objective = (2.0 * uplift) + (0.5 * combo_sh) - (0.08 * abs(min(mdd, 0.0)))

                        combo_rows.append({
                            "lookback": lookback,
                            "entry_z": float(entry_z),
                            "exit_z": float(exit_z),
                            "w_stoch": float(w_stoch),
                            "w_ratio": float(w_ratio),
                            "ratio_sharpe": float(ratio_sh) if np.isfinite(ratio_sh) else np.nan,
                            "baseline_sharpe": float(baseline_sharpe),
                            "combo_sharpe": float(combo_sh),
                            "uplift_vs_stoch": float(uplift),
                            "max_drawdown": float(mdd),
                            "objective": float(objective),
                        })

                        if objective > best_combo["objective"]:
                            best_combo = {
                                "objective": float(objective),
                                "sharpe": float(combo_sh),
                                "uplift": float(uplift),
                                "params": {
                                    "stoch": best_stoch["params"],
                                    "ratio": {
                                        "lookback": int(lookback),
                                        "entry_z": float(entry_z),
                                        "exit_z": float(exit_z),
                                    },
                                    "weights": {
                                        "w_stoch": float(w_stoch),
                                        "w_ratio": float(w_ratio),
                                    },
                                },
                                "returns": combo_ret,
                                "weights": combo_weight,
                                "stoch_returns": best_stoch["returns"],
                                "stoch_sharpe": float(baseline_sharpe),
                                "ratio_returns": ratio_ret,
                                "ratio_sharpe": float(ratio_sh) if np.isfinite(ratio_sh) else np.nan,
                            }

    stoch_df = pd.DataFrame(stoch_rows).sort_values("sharpe", ascending=False)
    combo_df = pd.DataFrame(combo_rows).sort_values(["objective", "combo_sharpe"], ascending=False)
    return stoch_df, combo_df, best_combo



In [ ]:
stoch_search_df, combo_search_df, best = optimize_linear_combo(df)
if best["params"] is None:
    raise RuntimeError("No valid combo found")

btc = df["btc"]
eth = df["eth"]

# Rebuild component returns from selected params for explicit comparison
sp = best["params"]["stoch"]
rp = best["params"]["ratio"]
wp = best["params"]["weights"]

stoch_pos = build_stochastic_signal(btc, k_window=sp["k_window"], d_window=sp["d_window"])
ratio_pos, ratio_spread = build_ratio_signal(btc, eth, lookback=rp["lookback"], entry_z=rp["entry_z"], exit_z=rp["exit_z"])

stoch_ret = backtest_weighted_long(btc, stoch_pos)
ratio_ret = backtest_weighted_long(btc, ratio_pos)
combo_weight = (wp["w_stoch"] * stoch_pos + wp["w_ratio"] * ratio_pos).clip(upper=2.5)
combo_ret = backtest_weighted_long(btc, combo_weight)

btc_ret = btc.pct_change().fillna(0.0)
eth_ret = eth.pct_change().fillna(0.0)

metrics = pd.DataFrame([
    summarize(stoch_ret, "BTC Stochastic (best baseline)"),
    summarize(ratio_ret, "BTC/ETH Ratio"),
    summarize(combo_ret, "Linear Combo (Stoch + Ratio)"),
    summarize(btc_ret, "BTC Buy & Hold"),
    summarize(eth_ret, "ETH Buy & Hold"),
]).set_index("strategy")

print("Best stochastic baseline params:", sp)
print("Best ratio params:", rp)
print("Best combo weights:", wp)
print("Best objective:", round(best["objective"], 4))
print("Baseline stochastic Sharpe:", round(metrics.loc["BTC Stochastic (best baseline)", "sharpe"], 4))
print("Combo Sharpe:", round(metrics.loc["Linear Combo (Stoch + Ratio)", "sharpe"], 4))
print("Sharpe uplift vs stochastic:", round(metrics.loc["Linear Combo (Stoch + Ratio)", "sharpe"] - metrics.loc["BTC Stochastic (best baseline)", "sharpe"], 4))
print(metrics[["sharpe", "annualized_return", "annualized_vol", "max_drawdown", "calmar"]].round(4))

combo_sharpe = metrics.loc["Linear Combo (Stoch + Ratio)", "sharpe"]
baseline_sharpe = metrics.loc["BTC Stochastic (best baseline)", "sharpe"]
if np.isfinite(combo_sharpe) and np.isfinite(baseline_sharpe):
    if combo_sharpe > baseline_sharpe:
        print("Great: combo Sharpe is above stochastic baseline.")
    else:
        print("Combo Sharpe did not beat stochastic baseline; ratio model may not add value on this sample.")



In [ ]:
eq = pd.DataFrame({
    "BTC Stochastic": (1 + stoch_ret).cumprod(),
    "BTC/ETH Ratio": (1 + ratio_ret).cumprod(),
    "Linear Combo (Stoch + Ratio)": (1 + combo_ret).cumprod(),
    "BTC Buy & Hold": (1 + btc_ret).cumprod(),
    "ETH Buy & Hold": (1 + eth_ret).cumprod(),
}).dropna()

fig, axes = plt.subplots(2, 1, figsize=(12, 9), sharex=False)

for col in eq.columns:
    lw = 2.2 if col == "Linear Combo (Stoch + Ratio)" else 1.4
    axes[0].plot(eq.index, eq[col], label=col, linewidth=lw)

axes[0].set_yscale("log")
axes[0].set_title("Cumulative Growth (Log Scale)")
axes[0].set_ylabel("Growth of $1")
axes[0].grid(True, alpha=0.3)
axes[0].legend(loc="best")

sh = metrics["sharpe"].sort_values(ascending=False)
x = np.arange(len(sh))
axes[1].bar(x, sh.values)
axes[1].set_xticks(x)
axes[1].set_xticklabels(sh.index, rotation=20, ha="right")
axes[1].set_title("Sharpe Ratio Comparison")
axes[1].set_ylabel("Sharpe")
axes[1].grid(True, axis="y", alpha=0.3)

plt.tight_layout()
plt.show()
